In [1]:
# convert 3d data to labeled image data
import cv2
import numpy as np
from matplotlib import pyplot as plt
import time
import datetime
from pathlib import Path
import glob
import json
import math
import h5py

with open("./Our_images/Capstone/our_images_labaled_data.json", "r") as outfile:
    pointcloud_data = json.load(outfile)
        
# with open("our_labaled_data.json", "r") as outfile:
#     our_data = json.load(outfile)
    
# pointcloud_data.update(our_data)

classes = classes = ['LookLeft', 'LookRight', 'LookUp', 'Neutral', 'RaiseEyebrows', 'Smile', 'OpenMouth']

images_path = list(pointcloud_data.keys())
# display(images_path[0])

#array of labeled face image data
face_images = []

#array of labels
labels = []

# How much, in terms of percentage of image, do we want to add to the side of the face
EXTRA_PADDING=0.02
IMG_SIZE=255
#create or load face image data
file_name = "./Data/all_labeled_images_data.h5"

try:
    file = h5py.File(file_name, "w")
except:
    file = h5py.File(file_name, "r")

for image_path in images_path:
    image = cv2.imread(image_path)
    try:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    except:
        continue
    dimensions = image.shape
    
    # if grey scale wanted
    # gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    height = image.shape[0]
    width = image.shape[1]
    channels = image.shape[2]

    #mediapipe mesh points used to get face bounding box
    points = np.array(pointcloud_data.get(image_path)[0])

    min_x = (min(points[:, 0])-EXTRA_PADDING) if (min(points[:, 0])-EXTRA_PADDING) > 0 else 0
    max_x = (max(points[:, 0])+EXTRA_PADDING) if (min(points[:, 0])+EXTRA_PADDING) < 1 else 1
    min_y = (min(points[:, 1])-EXTRA_PADDING) if (min(points[:, 0])-EXTRA_PADDING) > 0 else 0
    max_y = (max(points[:, 1])+EXTRA_PADDING) if (min(points[:, 0])+EXTRA_PADDING) < 1 else 1

    #change from ratio to coordinate
    face_Coor_min_x = int((min_x)*width)
    face_Coor_max_x = int((max_x)*width)
    face_Coor_min_y = int((min_y)*height)
    face_Coor_max_y = int((max_y)*height)

    #square up the face crop
    if (face_Coor_max_x-face_Coor_min_x) < (face_Coor_max_y-face_Coor_min_y):
        face_Coor_max_x=math.ceil((face_Coor_max_x+face_Coor_min_x)/2)+math.ceil((face_Coor_max_y-face_Coor_min_y)/2)
        face_Coor_min_x=math.floor((face_Coor_max_x+face_Coor_min_x)/2)-math.ceil((face_Coor_max_y-face_Coor_min_y)/2)
        if face_Coor_min_x < 0 :
            face_Coor_min_x = 0
        if face_Coor_max_x > width:
            face_Coor_max_x = width
    elif (face_Coor_max_x-face_Coor_min_x) > (face_Coor_max_y-face_Coor_min_y):
        face_Coor_max_y=math.ceil((face_Coor_max_y+face_Coor_min_y)/2)+math.ceil((face_Coor_max_x-face_Coor_min_x)/2)
        face_Coor_min_y=math.floor((face_Coor_max_y+face_Coor_min_y)/2)-math.ceil((face_Coor_max_x-face_Coor_min_x)/2)
        if face_Coor_min_y < 0 :
            face_Coor_min_y = 0
        if face_Coor_max_y > height:
            face_Coor_max_y = height

    #make face crops completely square
    count =0
    while (face_Coor_max_x-face_Coor_min_x) != (face_Coor_max_y-face_Coor_min_y):
        if (face_Coor_max_x-face_Coor_min_x) < (face_Coor_max_y-face_Coor_min_y):
            if count%2==0:
                face_Coor_max_x+=1
            elif count%2==1:
                face_Coor_min_x-=1
        elif (face_Coor_max_x-face_Coor_min_x) > (face_Coor_max_y-face_Coor_min_y):
            if count%2==0:
                face_Coor_max_y+=1
            elif count%2==1:
                face_Coor_min_y-=1
        count+=1
    
    #Crop face out of image
    face = image[face_Coor_min_y:face_Coor_max_y, face_Coor_min_x:face_Coor_max_x]

    # Final dimentions of classified face images
    dim = (IMG_SIZE, IMG_SIZE)

    #Resize face to specified dimentions
    resized = cv2.resize(face, dim, interpolation = cv2.INTER_AREA)
    face_images.append(resized)
    labels.append(classes.index(pointcloud_data.get(image_path)[1]))
    
dataset = file.create_dataset(
        "images", np.shape(face_images), h5py.h5t.STD_U8BE, data=face_images
    )

labels_set = file.create_dataset(
        "labels", np.shape(labels), h5py.h5t.STD_U8BE, data=labels
    )

file.close()